# Data

## Crowd human

In [ ]:
from pathlib import Path

data_dir = Path("/home/username/Downloads/crowdhuman")

In [ ]:
import pandas as pd

train_crowd_file = Path("../../../mlops/crowdhuman/annotation_train.csv")
train_df = pd.read_csv(train_crowd_file)
train_df.head(1)

In [ ]:
val_crowd_file = Path("../../../mlops/crowdhuman/annotation_val.csv")
val_df = pd.read_csv(val_crowd_file)
val_df.head(1)

In [ ]:
train_df.shape

In [ ]:
import albumentations as A
import albumentations.augmentations.geometric.functional as F
import cv2

class AspectRatioResize(A.DualTransform):
    """Resize the input to the given height and width.

    Args:
        p (float): probability of applying the transform. Default: 1.
        height (int): desired height of the output.
        width (int): desired width of the output.
        interpolation (OpenCV flag): flag that is used to specify the interpolation algorithm. Should be one of:
            cv2.INTER_NEAREST, cv2.INTER_LINEAR, cv2.INTER_CUBIC, cv2.INTER_AREA, cv2.INTER_LANCZOS4.
            Default: cv2.INTER_LINEAR.

    Targets:
        image, mask, bboxes

    Image types:
        uint8, float32
    """

    def __init__(self, height, width, interpolation=cv2.INTER_LINEAR, always_apply=False, p=1):
        super(AspectRatioResize, self).__init__(always_apply, p)
        self.height = height
        self.width = width
        self.interpolation = interpolation

    def apply(self, img, interpolation=cv2.INTER_LINEAR, **params):
        height, width, _ = img.shape
        r = min(self.width / width, self.height / height)
        return F.resize(
            img, height=int(r * height), width=int(r * width), interpolation=interpolation
        )

    def apply_to_bbox(self, bbox, **params):
        # Bounding box coordinates are scale invariant
        return bbox

In [ ]:
import albumentations as A

# IMAGE_SIZE = (416, 416)
IMAGE_SIZE = (1088, 608)
# IMAGE_SIZE = (608, 608)
train_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT,
            value=[128, 128, 128]
        ),
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
        A.HorizontalFlip(p=0.5),
        A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, val_shift_limit=20, p=0.5),
    ], 
    p=1.0, 
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=[]
    )
)

val_transform = A.Compose(
    [
        AspectRatioResize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0),
        A.PadIfNeeded(
            min_height=IMAGE_SIZE[1], min_width=IMAGE_SIZE[0], border_mode=cv2.BORDER_CONSTANT,
            value=[128, 128, 128]
        )
#         A.Resize(height=IMAGE_SIZE[1], width=IMAGE_SIZE[0], p=1.0)
    ],
    p=1.0,
    bbox_params=A.BboxParams(
        format="pascal_voc",
        min_area=0,
        min_visibility=0,
        label_fields=[]
    )
)

In [ ]:
from src.train_dataset import TrainDataset
%load_ext autoreload
%autoreload 2

train_dir = Path(data_dir, "Images")
train_dataset = TrainDataset(train_dir, train_df, train_transform)
val_dataset = TrainDataset(train_dir, val_df, val_transform)

In [ ]:
len(train_dataset), len(val_dataset)

In [ ]:
# train_dataset.data_groups = train_dataset.data_groups[:1000]
# val_dataset.data_groups = val_dataset.data_groups[:1000]

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
import cv2

image, centers, dimensions = train_dataset[0].values()
for center, dimension in zip(centers, dimensions):
    x, y = map(int, center)
    l, t, r, b = map(int, dimension)
    cv2.rectangle(
        image, (x - l, y - t), (x + r, y + b), (1.0, 0.0, 0.0), 2
    )

plt.figure()
plt.imshow(image)
plt.show()

# Modeling

In [ ]:
# !pip install gdown

In [ ]:
# import gdown

# gdown.download(
#     "https://drive.google.com/uc?id=1SFOhg_vos_xSYHLMTDGFVZBYjo8cr2fG", str(Path(data_dir, "crowdhuman_dla34.pth")), quiet=False
# )
# gdown.download(
#     "https://drive.google.com/uc?id=1iqRQjsG9BawIl8SlFomMg5iwkb6nqSpi", str(Path(data_dir, "fairmot_dla34.pth")), quiet=False
# )

In [ ]:
!mkdir -p checkpoint log

In [ ]:
from src.pose_dla_dcn import get_pose_net as get_dla_dcn
from src.trainer import Trainer
%load_ext autoreload
%autoreload 2

num_layers = 34
heads = {"hm": 1, "wh": 4, "reg": 2}
head_conv = 256
net = get_dla_dcn(num_layers, heads, head_conv)
model_path = Path(data_dir, "crowdhuman_dla34.pth")
# model_path = Path(data_dir, "fairmot_dla34.pth")
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda:1", checkpoint_dir="checkpoint", log_dir="log",
    model_path=None
)

In [ ]:
trainer.post_processor.conf_threshold

In [ ]:
trainer.fit(train_dataset, val_dataset, batch_size=16, start_epoch=0, end_epoch=3)

In [ ]:
trainer.fit(train_dataset, val_dataset, batch_size=16, start_epoch=3, end_epoch=5)

In [ ]:
trainer.fit(train_dataset, val_dataset, batch_size=16, start_epoch=5, end_epoch=6)

In [ ]:
trainer.post_processor.conf_threshold = 0.5

In [ ]:
trainer.optimizer.param_groups[0]["lr"] = 1e-5
trainer.optimizer.param_groups[0]["weight_decay"] = 0

In [ ]:
trainer.fit(train_dataset, val_dataset, batch_size=16, start_epoch=6, end_epoch=20)

In [ ]:
trainer.fit(train_dataset, val_dataset, batch_size=16, start_epoch=20, end_epoch=40)

In [ ]:
import torch
trainer.net.load_state_dict(torch.load(Path(data_dir, "crowdhuman_dla34.pth"))["state_dict"], strict=False)

In [ ]:
trainer = Trainer(
    net, image_size=IMAGE_SIZE, device="cuda:1", checkpoint_dir="checkpoint", log_dir="log",
    model_path=Path(data_dir, "crowdhuman_dla34.pth")
#     model_path="checkpoint/39.pth"
)

In [ ]:
trainer.eval(val_dataset, batch_size=16)